In [67]:
import pandas as pd
from functions.word_preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import pickle
import datetime
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import csv


pd.set_option('display.max_colwidth', 500)

In [2]:
directory = '/Users/kwheatley/Desktop/Capstone/gcloud_data/'

In [3]:
resume_edu = pd.read_csv(directory+'00_resumes%2Fprocessed_resumes_education.csv',header=None)
resume_edu.columns = ['city','resume_id','container_type','edu_title',
                      'edu_school','edu_addressLocality','edu_dates']

resume_work = pd.read_csv(directory+'02_processed_resumes_work_CLEAN.csv')

In [5]:
resume_edu = resume_edu[~resume_edu.edu_title.isnull()]
data = resume_edu

In [7]:
data.loc[data.edu_dates.isnull(),'edu_dates'] = 'January 1900'
data['from_date'] = data.edu_dates.apply(lambda x: (x.split(" to ")[0]).strip())
data['from_date'] = data.from_date.apply(lambda x: 
                        datetime.datetime.strptime('January ' + x, '%B %Y')
                        if len(x.split(" "))==1
                        else datetime.datetime.strptime(x, '%B %Y'))
data['to_date'] = data.edu_dates.apply(lambda x: (x.split(" to ")[1]).strip()
                        if len(x.split(" to "))>1
                        else x.strip())
data['currently_here'] = 'No'
data.loc[data.to_date == 'Present','currently_here'] = 'Yes'
data.loc[data.to_date == 'Present','to_date'] = 'July 2018'
data['to_date'] = data.to_date.apply(lambda x: 
                        datetime.datetime.strptime('December ' + x, '%B %Y')
                        if len(x.split(" "))==1
                        else datetime.datetime.strptime(x, '%B %Y'))
data['from_year'] = data.from_date.apply(lambda x: str(x.year))
data['to_year'] = data.to_date.apply(lambda x: str(x.year))
data['days_edu'] = (data.to_date - data.from_date).dt.days
data['years_edu'] = round(data.days_edu/365)

In [18]:
edu_titles = data.edu_title
edu_titles = [re.sub('[^A-Za-z0-9\s]+', '', row.lower()) for row in edu_titles]

edu_tokens = [nltk.word_tokenize(row) for row in edu_titles]

In [173]:
acronym_list = []
for row in edu_tokens:
    [acronym_list.append(x) for x in row if len(x) < 5]
acronym_list = list(set(acronym_list))

degrees = ['associates', 'associate', 'association', 'associated', 'associatge ', 'assoc',
           'bachelors', 'bachelor', 
           'science', 'sciences',
           'masters', 'master',
           'doctor', 'baccalaureate', 'baccalaurat', 'baccalaureat', 'bechelor',
           'certificate', 'certificate', 'certification', 'certifcate', 'certifed', 'certified', 
           'certificantion', 'certificates',
           'doctorate', 
           'juris', 'license', 'minor',
           'achievement', 'achievements', 'advanced', 'regents',
           'diploma','college', 'junior', 'applied', 'vocational', 'school', 'degree',
           'laude',
          ]

degree_name_list = []
subject_name_list = []
for row in edu_tokens:
    
    degree_row = []
    subject_row = row
    for token in row:
        if token == 'in':
            degree_row.append(token)
            subject_row = subject_row[1:]
            break
        elif token in degrees + acronym_list:
            degree_row.append(token)
            subject_row = subject_row[1:]
        else:
            break

    degree_name_list.append(' '.join(degree_row))
    subject_name_list.append(' '.join(subject_row))

final = list(zip(degree_name_list, subject_name_list))
pd.DataFrame(list(set(final))).to_csv('final.csv')
pd.DataFrame(list(set(degree_name_list))).to_csv('degree_name_list.csv')

In [168]:
counter_temp = Counter(degree_name_list)    
writefile = open('test1.csv', 'w')
writer = csv.writer(writefile)
for key, count in counter_temp.items():
    writer.writerow([str(key),str(count)])
writefile.close()

pd.DataFrame(list(set(subject_name_list))).to_csv('temp.csv')

In [ ]:
# ngram_counter = Counter()

for row in edu_tokens:
    unigrams = ngrams(row,1)
#     bigrams = ngrams(row,2)
#     tigrams = ngrams(row,3)

    ngram_counter.update(unigrams)
#     ngram_counter.update(bigrams)
#     ngram_counter.update(tigrams) 

In [61]:
# ngram_counter.most_common(500)
unigram_list = sorted([word[0] for word in list(set(ngram_counter.elements()))])
pd.DataFrame(unigram_list).to_csv('unigram_list.csv')

In [11]:
# for i in resume_work.groupby(['role','cleaned_job_title','experiences','currently_here'])\
# ['days_worked'].agg('mean').reset_index()['experiences'].unique():
#     print(i)
# resume_work[resume_work.experiences == '4, 7'].role

In [ ]:
# https://developers.google.com/chart/interactive/docs/gallery/sankey